# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [51]:
# Import libraries
import numpy as np
import pandas as pd

In [52]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [53]:
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = sum(student_data['passed'] == 'yes')
n_failed = sum(student_data['passed'] == 'no')
grad_rate = (100.0 * n_passed) / n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [54]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [55]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [56]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

training_indices = np.random.choice(student_data.index.values, num_train, replace=False)

X_train = X_all.ix[training_indices]
y_train = y_all.ix[training_indices]
X_test = X_all.drop(training_indices)
y_test = y_all.drop(training_indices)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
assert len(student_data) == len(X_train) + len(X_test) == len(y_train) + len(y_test)

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [57]:
# HELPER FUNCTIONS: TRAINING, TESTING (F1), LEARNING CURVE
import time
from sklearn.metrics import f1_score, make_scorer
from functools import partial

def train_classifier(clf, X_train, y_train, verbose=False):
    """
    Train a generic sklearn classifier $clf using $X_train and $y_train.
    Mutates clf. Returns the amount of time spent training.
    """
    if verbose:
        print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    training_time = end - start
    if verbose:
        print "Done!\nTraining time (secs): {:.3f}".format(training_time)
    return training_time

def predict_labels(clf, features, target, verbose=False):
    """
    Use classifier $clf to predict target labels using $features, and compute the F1 score wrt $target.
    Return the F1 score and the time spent predicting labels.
    """
    if verbose:
        print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    predicting_time = end - start
    if verbose:
        print "Done!\nPrediction time (secs): {:.3f}".format(predicting_time)
    return f1_score(target.values, y_pred, pos_label='yes'), predicting_time

# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test, verbose=False):
    if verbose:
        print "------------------------------------------"
        print "Training set size: {}".format(len(X_train))
    training_time = train_classifier(clf, X_train, y_train, verbose)
    training_f1, _ = predict_labels(clf, X_train, y_train, verbose)
    testing_f1, predicting_time = predict_labels(clf, X_test, y_test, verbose)
    if verbose:
        print "F1 score for training set: {}".format(training_f1)
        print "F1 score for test set: {}".format(testing_f1)
    return (training_f1, training_time, testing_f1, predicting_time)

f1_score_yes = partial(f1_score, pos_label='yes')
f1_score_yes.__name__ = "f1 score (pos_label='yes')"

### Model Exploration

In [58]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from collections import defaultdict

num_repititions = 10
training_sizes = (100, 200, 300)
classifiers = {
    "DT": DecisionTreeClassifier(),
    "RF": RandomForestClassifier(),
    "kNN": KNeighborsClassifier(),
    "LDA": LinearDiscriminantAnalysis(),

}
performances = {c: {ts: defaultdict(list) for ts in training_sizes} for c in classifiers.keys()}
np.random.seed(123456)

for name, clf in classifiers.items():
    results = performances[name]
    for _ in xrange(num_repititions):
        for training_size in training_sizes:
            # Grab training_size samples from the training data
            training_indices = np.random.choice(X_train.index.values, training_size, replace=False)
            X_train_subset = X_train.ix[training_indices]
            y_train_subset = y_train.ix[training_indices]
            assert np.array_equal(X_train_subset.index, y_train_subset.index)
            training_f1, training_time, testing_f1, testing_time = train_predict(clf, X_train_subset, y_train_subset, X_test, y_test)
            results[training_size]['training_f1'].append(training_f1)
            results[training_size]['training_time'].append(training_time)
            results[training_size]['testing_f1'].append(testing_f1)
            results[training_size]['testing_time'].append(testing_time)

def print_performance_dict(p):
    for method, results in p.items():
        for training_size in sorted(results.keys()):
            print "-------------------"
            print "Training Size = {}".format(training_size)
            print "Average Training Time for {}: {:.3e}".format(method, np.mean(results[training_size]['training_time']))
            print "Average Training F1 for {}: {:.4}".format(method, np.mean(results[training_size]['training_f1']))
            print "Average Testing (Prediction) Time for {}: {:.3e}".format(method, np.mean(results[training_size]['testing_time']))
            print "Average Testing F1 for {}: {:.4}".format(method, np.mean(results[training_size]['testing_f1']))
            print

print_performance_dict(performances)

-------------------
Training Size = 100
Average Training Time for kNN: 5.019e-04
Average Training F1 for kNN: 0.8351
Average Testing (Prediction) Time for kNN: 1.470e-03
Average Testing F1 for kNN: 0.7798

-------------------
Training Size = 200
Average Training Time for kNN: 5.607e-04
Average Training F1 for kNN: 0.8503
Average Testing (Prediction) Time for kNN: 1.916e-03
Average Testing F1 for kNN: 0.7552

-------------------
Training Size = 300
Average Training Time for kNN: 6.866e-04
Average Training F1 for kNN: 0.8557
Average Testing (Prediction) Time for kNN: 2.541e-03
Average Testing F1 for kNN: 0.7844

-------------------
Training Size = 100
Average Training Time for DT: 6.451e-04
Average Training F1 for DT: 1.0
Average Testing (Prediction) Time for DT: 1.502e-04
Average Testing F1 for DT: 0.7394

-------------------
Training Size = 200
Average Training Time for DT: 1.092e-03
Average Training F1 for DT: 1.0
Average Testing (Prediction) Time for DT: 1.525e-04
Average Testing F1 

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [61]:
from sklearn.grid_search import GridSearchCV
from sklearn.externals.six import StringIO
from sklearn import tree
import os

dt_params = [
    {
        'criterion': ['gini', 'entropy'],
        'max_depth': range(1, 21),
        'min_samples_split': range(1, 10),
        'min_samples_leaf': range(1, 10)
    }
]

make_plots = False

# Use n=300 observations used by previous models
dt_clf = GridSearchCV(DecisionTreeClassifier(), dt_params, cv=5, scoring=make_scorer(f1_score_yes))
train_predict(dt_clf, X_train, y_train, X_test, y_test, True)
dt_final = dt_clf.best_estimator_

if make_plots:
    with open("dt1.dot", 'w') as f:
        f = tree.export_graphviz(dt_final, out_file=f)
    !dot -Tpdf dt1.dot -o dt1.pdf
    os.unlink('dt1.dot')


# Use all available data (n=395)
dt_clf2 = GridSearchCV(DecisionTreeClassifier(), dt_params, cv=5, scoring=make_scorer(f1_score_yes))
train_classifier(dt_clf2, X_all, y_all, True)
dt_final2 = dt_clf2.best_estimator_

if make_plots:
    with open("dt2.dot", 'w') as f:
        f = tree.export_graphviz(dt_final2, out_file=f)
    !dot -Tpdf dt2.dot -o dt2.pdf
    os.unlink('dt2.dot')

------------------------------------------
Training set size: 300
Training GridSearchCV...
Done!
Training time (secs): 69.853
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.000
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.806451612903
F1 score for test set: 0.825174825175
Training GridSearchCV...
Done!
Training time (secs): 75.598


In [64]:
# See the results
print "n=300 best params:"
print dt_clf.best_params_
print "n=300 best CV score:"
print dt_clf.best_score_
print

print "n=395 best params:"
print dt_clf2.best_params_
print "n=395 best CV score:" 
print dt_clf2.best_score_
print

# Which were the important variables?
nonzero_indices = [i for i, score in enumerate(dt_final2.feature_importances_) if score > 0]
print "Variables used for splits:" 
print X_all.columns[nonzero_indices]


n=300 best params:
{'min_samples_split': 1, 'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1}
n=300 best CV score:
0.806536056376

n=395 best params:
{'min_samples_split': 1, 'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1}
n=395 best CV score:
0.810955914371

Variables used for splits:
Index([u'failures'], dtype='object')
